# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [2]:
import os
import sys

sys.path.append('..')
from utils import *

# Set your preferred name
USER = "Jiayi"
# Set the data and output directories
DATA_ROOT = '/Users/legion/Desktop/Courses/IS389/data'   
OUTPUT_ROOT = '/Users/legion/Desktop/Courses/IS389/output'
TRAIT = 'Hepatitis'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [ ]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [ ]:
dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

If no match is found, jump directly to GEO in Part 2.2

In [ ]:
trait_subdir = "TCGA_Adrenocortical_Cancer_(ACC)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.ACC.sampleMap_ACC_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.ACC.sampleMap_HiSeqV2_PANCAN.gz')

In [ ]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [ ]:
def check_rows_and_columns(dataframe, display=False):
    """
    Get the lists of row names and column names of a dataset, and optionally observe them.
    :param dataframe:
    :param display:
    :return:
    """
    dataframe_rows = dataframe.index.tolist()
    if display:
        print(f"The dataset has {len(dataframe_rows)} rows, such as {dataframe_rows[:20]}")
    dataframe_cols = dataframe.columns.tolist()
    if display:
        print(f"\nThe dataset has {len(dataframe_cols)} columns, such as {dataframe_cols[:20]}")
    return dataframe_rows, dataframe_cols

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)
clinical_data_cols[:10]

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

In [ ]:
candidate_age_cols = [ 'age_at_initial_pathologic_diagnosis',
                      'days_to_birth', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = [ 'gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
def preview_df(df, n=5):
    return df.head(n).to_dict(orient='list')

In [ ]:
preview_df(clinical_data[candidate_age_cols])

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

In [ ]:
gender_col = 'gender'

In [ ]:
def xena_select_clinical_features(clinical_df, trait, age_col=None, gender_col=None):
    feature_list = []
    trait_data = clinical_df.index.to_series().apply(xena_convert_trait).rename(trait)
    feature_list.append(trait_data)
    if age_col:
        age_data = clinical_df[age_col].apply(xena_convert_age).rename("Age")
        feature_list.append(age_data)
    if gender_col:
        gender_data = clinical_df[gender_col].apply(xena_convert_gender).rename("Gender")
        feature_list.append(gender_data)
    selected_clinical_df = pd.concat(feature_list, axis=1)
    return selected_clinical_df

In [ ]:
def xena_convert_trait(row_index: str):
    """
    Convert the trait information from Sample IDs to labels depending on the last two digits.
    Tumor types range from 01 - 09, normal types from 10 - 19.
    :param row_index: the index value of a row
    :return: the converted value
    """
    last_two_digits = int(row_index[-2:])

    if 1 <= last_two_digits <= 9:
        return 1
    elif 10 <= last_two_digits <= 19:
        return 0
    else:
        return -1

In [ ]:
def xena_convert_age(cell: str):
    """Convert the cell content about age to a numerical value using regular expression
    """
    match = re.search(r'\d+', str(cell))
    if match:
        return int(match.group())
    else:
        return None

In [ ]:
def xena_convert_gender(cell: str):
    """Convert the cell content about gender to a binary value
    """
    if isinstance(cell, str):
        cell = cell.lower()

    if cell == "female":
        return 0
    elif cell == "male":
        return 1
    else:
        return None

In [ ]:
import re
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col=age_col, gender_col=gender_col)

In [ ]:
def normalize_gene_symbols_in_index(gene_df):
    """Normalize the human gene symbols at the index of a dataframe, and replace the index with its normalized version.
    Remove the rows where the index failed to be normalized."""
    normalized_gene_list = normalize_gene_symbols(gene_df.index.tolist())
    assert len(normalized_gene_list) == len(gene_df.index)
    gene_df.index = normalized_gene_list
    gene_df = gene_df[gene_df.index.notnull()]
    return gene_df

In [ ]:
def normalize_gene_symbols(gene_symbols, batch_size=1000):
    """Normalize human gene symbols in batches using the 'mygenes' library"""
    mg = mygene.MyGeneInfo()
    normalized_genes = {}

    # Process in batches
    for i in range(0, len(gene_symbols), batch_size):
        batch = gene_symbols[i:i + batch_size]
        results = mg.querymany(batch, scopes='symbol', fields='symbol', species='human')

        # Update the normalized_genes dictionary with results from this batch
        for gene in results:
            normalized_genes[gene['query']] = gene.get('symbol', None)

    # Return the normalized symbols in the same order as the input
    return [normalized_genes.get(symbol) for symbol in gene_symbols]


In [ ]:
import mygene

if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

In [ ]:
def judge_and_remove_biased_features(df, trait, trait_type):
    assert trait_type in ["binary", "continuous"], f"The trait must be either a binary or a continuous variable!"
    if trait_type == "binary":
        trait_biased = judge_binary_variable_biased(df, trait)
    else:
        trait_biased = judge_continuous_variable_biased(df, trait)
    if trait_biased:
        print(f"The distribution of the feature \'{trait}\' in this dataset is severely biased.\n")
    else:
        print(f"The distribution of the feature \'{trait}\' in this dataset is fine.\n")
    if "Age" in df.columns:
        age_biased = judge_continuous_variable_biased(df, 'Age')
        if age_biased:
            print(f"The distribution of the feature \'Age\' in this dataset is severely biased.\n")
            df = df.drop(columns='Age')
        else:
            print(f"The distribution of the feature \'Age\' in this dataset is fine.\n")
    if "Gender" in df.columns:
        gender_biased = judge_binary_variable_biased(df, 'Gender')
        if gender_biased:
            print(f"The distribution of the feature \'Gender\' in this dataset is severely biased.\n")
            df = df.drop(columns='Gender')
        else:
            print(f"The distribution of the feature \'Gender\' in this dataset is fine.\n")

    return trait_biased, df

In [ ]:
def judge_binary_variable_biased(dataframe, col_name, min_proportion=0.1, min_num=5):
    """
    Check if the distribution of a binary variable in the dataset is too biased to be usable for analysis
    :param dataframe:
    :param col_name:
    :param min_proportion:
    :param min_num:
    :return:
    """
    label_counter = dataframe[col_name].value_counts()
    total_samples = len(dataframe)
    rare_label_num = label_counter.min()
    rare_label = label_counter.idxmin()
    rare_label_proportion = rare_label_num / total_samples

    print(
        f"For the feature \'{col_name}\', the least common label is '{rare_label}' with {rare_label_num} occurrences. This represents {rare_label_proportion:.2%} of the dataset.")

    biased = (len(label_counter) < 2) or ((rare_label_proportion < min_proportion) and (rare_label_num < min_num))
    return bool(biased)


In [ ]:
def judge_continuous_variable_biased(dataframe, col_name):
    """Check if the distribution of a continuous variable in the dataset is too biased to be usable for analysis.
    As a starting point, we consider it biased if all values are the same. For the next step, maybe ask GPT to judge
    based on quartile statistics combined with its common sense knowledge about this feature.
    """
    quartiles = dataframe[col_name].quantile([0.25, 0.5, 0.75])
    min_value = dataframe[col_name].min()
    max_value = dataframe[col_name].max()

    # Printing quartile information
    print(f"Quartiles for '{col_name}':")
    print(f"  25%: {quartiles[0.25]}")
    print(f"  50% (Median): {quartiles[0.5]}")
    print(f"  75%: {quartiles[0.75]}")
    print(f"Min: {min_value}")
    print(f"Max: {max_value}")

    biased = min_value == max_value

    return bool(biased)

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

In [ ]:
from typing import Callable, Optional, List, Tuple, Union, Any

In [ ]:
def save_cohort_info(cohort: str, info_path: str, is_available: bool, is_biased: Optional[bool] = None,
                     df: Optional[pd.DataFrame] = None, note: str = '') -> None:
    """
    Add or update information about the usability and quality of a dataset for statistical analysis.

    Parameters:
    cohort (str): A unique identifier for the dataset.
    info_path (str): File path to the JSON file where records are stored.
    is_available (bool): Indicates whether both the genetic data and trait data are available in the dataset, and can be
     preprocessed into a dataframe.
    is_biased (bool, optional): Indicates whether the dataset is too biased to be usable.
        Required if `is_available` is True.
    df (pandas.DataFrame, optional): The preprocessed dataset. Required if `is_available` is True.
    note (str, optional): Additional notes about the dataset.

    Returns:
    None: The function does not return a value but updates or creates a record in the specified JSON file.
    """
    if is_available:
        assert (df is not None) and (is_biased is not None), "'df' and 'is_biased' should be provided if this cohort " \
                                                             "is relevant."
    is_usable = is_available and (not is_biased)
    new_record = {"is_usable": is_usable,
                  "is_available": is_available,
                  "is_biased": is_biased if is_available else None,
                  "has_age": "Age" in df.columns if is_available else None,
                  "has_gender": "Gender" in df.columns if is_available else None,
                  "sample_size": len(df) if is_available else None,
                  "note": note}
    
    if not os.path.exists(info_path):
        with open(info_path, 'w') as file:
            json.dump({}, file)
        print(f"A new JSON file was created at: {info_path}")

    with open(info_path, "r") as file:
        records = json.load(file)
    records[cohort] = new_record

    temp_path = info_path + ".tmp"
    try:
        with open(temp_path, 'w') as file:
            json.dump(records, file)
        os.replace(temp_path, info_path)

    except Exception as e:
        print(f"An error occurred: {e}")
        if os.path.exists(temp_path):
            os.remove(temp_path)
        raise

In [ ]:
import json

save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [3]:
dataset = 'GEO'
trait_subdir = "Hepatitis"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE113392', 'GSE113409', 'GSE140249', 'GSE78160', 'GSE85550']

Repeat the below steps for all the accession numbers

In [4]:
def get_relevant_filepaths(cohort_dir):
    """Find the file paths of a SOFT file and a matrix file from the given data directory of a cohort.
    If there are multiple SOFT files or matrix files, simply choose the first one. May be replaced by better
    strategies later.
    """
    files = os.listdir(cohort_dir)
    soft_files = [f for f in files if 'soft' in f.lower()]
    matrix_files = [f for f in files if 'matrix' in f.lower()]
    assert len(soft_files) > 0 and len(matrix_files) > 0
    soft_file_path = os.path.join(cohort_dir, soft_files[0])
    matrix_file_path = os.path.join(cohort_dir, matrix_files[0])

    return soft_file_path, matrix_file_path

In [5]:
# Stopped: No gene mapping
cohort = accession_num = "GSE140249"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

from utils import *
import io
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

clinical_data

!Series_title	"Distinct serum microRNA profiles in patients with active autoimmune hepatitis"
!Series_summary	"Serum microRNAs are useful for the early diagnosis and management of autoimmune hepatitis"
!Series_overall_design	"Serum microRNA profiles of patients with autoimmune hepatitis (AIH),  primary biliary cholangitis (PBC), overlap syndrome (OS), or without hepatic diseases."


,!Sample_geo_accession,GSM4158202,GSM4158203,GSM4158204,GSM4158205,GSM4158206,GSM4158207,GSM4158208,GSM4158209,GSM4158210,...,GSM4158334,GSM4158335,GSM4158336,GSM4158337,GSM4158338,GSM4158339,GSM4158340,GSM4158341,GSM4158342,GSM4158343
0,!Sample_characteristics_ch1,Sex: F,Sex: F,Sex: M,Sex: F,Sex: M,Sex: M,Sex: M,Sex: M,Sex: F,...,Sex: M,Sex: F,Sex: F,Sex: F,Sex: F,Sex: F,Sex: F,Sex: M,Sex: M,Sex: F
1,!Sample_characteristics_ch1,age: 32,age: 61,age: 74,age: 27,age: 72,age: 28,age: 71,age: 76,age: 38,...,age: 55,age: 48,age: 35,age: 34,age: 34,age: 23,age: 52,age: 54,age: 59,age: 69
2,!Sample_characteristics_ch1,disease state: AIH,disease state: AIH,disease state: AIH,disease state: AIH,disease state: AIH,disease state: AIH,disease state: AIH,disease state: AIH,disease state: AIH,...,disease state: Healthy,disease state: Healthy,disease state: Healthy,disease state: Healthy,disease state: Healthy,disease state: Healthy,disease state: Healthy,disease state: Healthy,disease state: Healthy,disease state: Healthy
3,!Sample_characteristics_ch1,treatment: Pre,treatment: Pre,treatment: Pre,treatment: Pre,treatment: Pre,treatment: Pre,treatment: Pre,treatment: Pre,treatment: Pre,...,treatment: N/A,treatment: N/A,treatment: N/A,treatment: N/A,treatment: N/A,treatment: N/A,treatment: N/A,treatment: N/A,treatment: N/A,treatment: N/A
4,!Sample_characteristics_ch1,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,...,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum


In [6]:
tumor_stage_row = clinical_data.iloc[2]
tumor_stage_row.unique()

array(['!Sample_characteristics_ch1', 'disease state: AIH',
       'disease state: PBC', 'disease state: OS',
       'disease state: Healthy'], dtype=object)

In [7]:
is_gene_availabe = True
trait_row = 2
age_row = 1
gender_row = 0

trait_type = 'binary'

# Verify and use the functions generated by GPT

# 这个函数将组织类型（tissue type）转换为有关癫痫存在与否的二进制值。
# 它是基于特定的假设，即如果组织类型是“胰腺导管腺癌”（Pancreatic Ductal Adenocarcinoma），则认为癫痫存在（返回1）；否则，认为癫痫不存在（返回0）。
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if (tissue_type == 'disease state: AIH' or tissue_type == 'disease state: OS'):
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present


# def convert_trait(tumor_grade):
#     if (tumor_grade == 'tumor grade: 2' or tumor_grade == 'tumor grade: 3' or tumor_grade == 'tumor grade: 4'):
#         return 1  
#     elif tumor_grade == 'tumor grade: 1':
#         return 0  
#     else:
#         return None

# 这个函数的目的是将年龄的字符串表示转换为一个连续的数值型表示。如果年龄未知（例如，标记为'n.a.'），则返回None。
# 函数尝试从传入的字符串中提取出一个整数作为年龄值。如果字符串的格式不符合预期，导致提取失败，同样返回None。
def convert_age(age_string):
    """
    Convert age string to a continuous numerical value.
    Unknown values are converted to None.
    """
    if age_string.lower() == 'n.a.':
        return None
    try:
        # Extract age as an integer from the string
        age = int(age_string.split(': ')[1])
        return age
    except (ValueError, IndexError):
        # In case of any format error or unexpected string structure
        return None


# 这个函数将性别的字符串表示转换为二进制值，其中“female”对应1，“male”对应0。如果性别未知或字符串不符合预期格式，则返回None。
# It sometimes maps 'female' to 0, and sometimes 1. Does it matter?
def convert_gender(gender_string):
    """
    Convert gender string to a binary value.
    'female' is represented as 1, 'male' as 0.
    Unknown values are converted to None.
    """
    if (gender_string.lower() == 'sex: female' or gender_string.lower() == 'sex: f' or gender_string.lower() == 'gender: female' or gender_string.lower() == 'gender: f'):
        return 1
    elif (gender_string.lower() == 'sex: male' or gender_string.lower() == 'sex: m' or gender_string.lower() == 'gender: male' or gender_string.lower() == 'gender: m') :  # changeed 
        return 0
    else:
        return None

In [8]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

c:\Users\legion\Desktop\TAIS_2024SP\TAIS_2024SP\Jiayi\..\utils.py:218: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
c:\Users\legion\Desktop\TAIS_2024SP\TAIS_2024SP\Jiayi\..\utils.py:218: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
c:\Users\legion\Desktop\TAIS_2024SP\TAIS_2024SP\Jiayi\..\utils.py:218: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)


,GSM4158202,GSM4158203,GSM4158204,GSM4158205,GSM4158206,GSM4158207,GSM4158208,GSM4158209,GSM4158210,GSM4158211,...,GSM4158334,GSM4158335,GSM4158336,GSM4158337,GSM4158338,GSM4158339,GSM4158340,GSM4158341,GSM4158342,GSM4158343
Hepatitis,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Age,32,61,74,27,72,28,71,76,38,41,...,55,48,35,34,34,23,52,54,59,69
Gender,1,1,0,1,0,0,0,0,1,1,...,0,1,1,1,1,1,1,0,0,1


In [9]:
genetic_data = get_genetic_data(matrix_file)
genetic_data

,GSM4158202,GSM4158203,GSM4158204,GSM4158205,GSM4158206,GSM4158207,GSM4158208,GSM4158209,GSM4158210,GSM4158211,...,GSM4158334,GSM4158335,GSM4158336,GSM4158337,GSM4158338,GSM4158339,GSM4158340,GSM4158341,GSM4158342,GSM4158343
ID,,,,,,,,,,,,,,,,,,,,,
MIMAT0000062,2.840,2.634,3.026,2.769,2.793,4.163,3.741,4.383,4.771,-4.833,...,3.863,3.344,4.191,4.145,4.225,3.173,3.030,4.526,2.904,6.561
MIMAT0000063,3.267,3.881,3.554,3.367,3.403,4.514,4.381,4.681,4.846,-3.565,...,4.468,4.054,4.552,4.003,4.497,4.079,4.087,4.730,4.321,6.585
MIMAT0000064,2.753,2.696,3.104,2.567,3.514,3.866,3.775,4.007,4.458,-4.833,...,3.674,3.512,4.011,3.327,4.058,3.185,3.097,4.573,3.593,6.395
MIMAT0000065,2.615,2.536,2.677,2.441,2.997,3.737,3.148,4.080,4.290,-4.833,...,3.232,3.245,3.758,2.931,3.865,2.710,2.767,4.173,2.648,6.301
MIMAT0000066,1.886,-6.084,1.181,0.947,-5.249,1.777,-2.601,-2.823,-3.993,-4.833,...,1.755,1.312,2.178,1.794,1.595,0.597,0.750,1.746,3.141,4.540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MIMAT0032116,6.093,5.952,5.927,5.501,5.784,5.053,5.563,5.417,5.693,5.021,...,4.469,5.376,4.720,4.396,4.723,4.477,5.014,4.756,5.304,5.340
MIMAT0033692,5.431,5.504,5.111,5.272,4.984,5.107,5.121,4.817,4.458,4.804,...,5.022,5.377,4.942,4.784,4.984,5.081,4.992,4.798,5.146,4.870
MIMAT0035542,4.919,4.165,4.908,4.289,4.876,3.714,4.365,3.654,4.777,3.617,...,3.027,3.948,2.701,3.691,3.332,2.330,3.160,3.361,3.546,3.391


In [10]:
requires_gene_mapping = True

if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    {gene_row_ids}
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')



{'ID': ['MIMAT0000062', 'MIMAT0000063', 'MIMAT0000064', 'MIMAT0000065', 'MIMAT0000066'], 'miRNA': ['MIMAT0000062', 'MIMAT0000063', 'MIMAT0000064', 'MIMAT0000065', 'MIMAT0000066'], 'miRNA_ID_LIST': ['hsa-let-7a-5p', 'hsa-let-7b-5p', 'hsa-let-7c-5p', 'hsa-let-7d-5p', 'hsa-let-7e-5p']}

    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    ['MIMAT0000062', 'MIMAT0000063', 'MIMAT0000064', 'MIMAT0000065', 'MIMAT0000066', 'MIMAT0000067', 'MIMAT0000068', 'MIMAT0000069', 'MIMAT0000070', 'MIMAT0000071', 'MIMAT0000072', 'MIMAT0000073', 'MIMAT0000074', 'MIMAT0000075', 'MIMAT0000076', 'MIMAT0000077', 'MIMAT0000078', 'MIMAT0000079', 'MIMAT0000080', 'MIMAT0000081']
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the id

Index(['ID', 'miRNA', 'miRNA_ID_LIST'], dtype='object')

In [11]:
gene_annotation.columns

Index(['ID', 'miRNA', 'miRNA_ID_LIST'], dtype='object')

In [ ]:
if requires_gene_mapping:
    identifier_key = 'ID'
    gene_symbol_key = 'Gene Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [13]:
# MemeoryError
cohort = accession_num = "GSE113409"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

from utils import *
import io
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

clinical_data

!Series_title	"Loci-specific differences in blood DNA methylation in HBV-negative populations at risk for hepatocellular carcinoma development - post-diagnostic HCC blood samples"
!Series_summary	"Genome wide DNA methylation profiling of blood samples collected from patients after diagnosis with hepatocellular carcinoma (HCC) (cases) vs. blood samples collected from healthy individuals without family history of cancer (controls). The Illumina Infinium 450K Human DNA methylation Beadchip v1.2 was used to obtain DNA methylation profiles across approximately 450,000 CpGs in human samples corresponding to cases (post-diagnostic HCC) and controls. Samples included 24 cases and 24 controls. Cases were matched with controls on gender, age, ethnicity, hepatitis C infection, and diabetes. The presence or absence of HCC in our study was determined based on the AASLD criteria."
!Series_overall_design	"Bisulphite converted DNA from the 42 samples were hybridised to the Illumina Infinium 450k Human

,!Sample_geo_accession,GSM3105174,GSM3105175,GSM3105176,GSM3105177,GSM3105178,GSM3105179,GSM3105180,GSM3105181,GSM3105182,...,GSM3105212,GSM3105213,GSM3105214,GSM3105215,GSM3105216,GSM3105217,GSM3105218,GSM3105219,GSM3105220,GSM3105221
0,!Sample_characteristics_ch1,case/control: Control - Healthy,case/control: Control - Healthy,case/control: Control - Healthy,case/control: Control - Healthy,case/control: Control - Healthy,case/control: Control - Healthy,case/control: Control - Healthy,case/control: Control - Healthy,case/control: Control - Healthy,...,case/control: Case - post-diagnostic hepatocel...,case/control: Case - post-diagnostic hepatocel...,case/control: Case - post-diagnostic hepatocel...,case/control: Case - post-diagnostic hepatocel...,case/control: Case - post-diagnostic hepatocel...,case/control: Case - post-diagnostic hepatocel...,case/control: Case - post-diagnostic hepatocel...,case/control: Case - post-diagnostic hepatocel...,case/control: Case - post-diagnostic hepatocel...,case/control: Case - post-diagnostic hepatocel...
1,!Sample_characteristics_ch1,disease state: normal,disease state: normal,disease state: normal,disease state: normal,disease state: normal,disease state: normal,disease state: normal,disease state: normal,disease state: normal,...,disease state: hepatocellular carcinoma (HCC),disease state: hepatocellular carcinoma (HCC),disease state: hepatocellular carcinoma (HCC),disease state: hepatocellular carcinoma (HCC),disease state: hepatocellular carcinoma (HCC),disease state: hepatocellular carcinoma (HCC),disease state: hepatocellular carcinoma (HCC),disease state: hepatocellular carcinoma (HCC),disease state: hepatocellular carcinoma (HCC),disease state: hepatocellular carcinoma (HCC)
2,!Sample_characteristics_ch1,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,...,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood


In [14]:
tumor_stage_row = clinical_data.iloc[1]
tumor_stage_row.unique()

array(['!Sample_characteristics_ch1', 'disease state: normal',
       'disease state: hepatocellular carcinoma (HCC)'], dtype=object)

In [15]:
is_gene_availabe = True
trait_row = 1
age_row = None
gender_row = None

trait_type = 'binary'

# Verify and use the functions generated by GPT

# 这个函数将组织类型（tissue type）转换为有关癫痫存在与否的二进制值。
# 它是基于特定的假设，即如果组织类型是“胰腺导管腺癌”（Pancreatic Ductal Adenocarcinoma），则认为癫痫存在（返回1）；否则，认为癫痫不存在（返回0）。
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'disease state: hepatocellular carcinoma (HCC)':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

In [16]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

c:\Users\legion\Desktop\TAIS_2024SP\TAIS_2024SP\Jiayi\..\utils.py:218: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)


,GSM3105174,GSM3105175,GSM3105176,GSM3105177,GSM3105178,GSM3105179,GSM3105180,GSM3105181,GSM3105182,GSM3105183,...,GSM3105212,GSM3105213,GSM3105214,GSM3105215,GSM3105216,GSM3105217,GSM3105218,GSM3105219,GSM3105220,GSM3105221
Hepatitis,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [17]:
genetic_data = get_genetic_data(matrix_file)
genetic_data

,GSM3105174,GSM3105175,GSM3105176,GSM3105177,GSM3105178,GSM3105179,GSM3105180,GSM3105181,GSM3105182,GSM3105183,...,GSM3105212,GSM3105213,GSM3105214,GSM3105215,GSM3105216,GSM3105217,GSM3105218,GSM3105219,GSM3105220,GSM3105221
ID,,,,,,,,,,,,,,,,,,,,,
cg00000029,0.456141,0.545553,0.491754,0.549807,0.512640,0.627964,0.430656,0.571047,0.499600,0.532660,...,0.496885,0.548694,0.511098,0.618431,0.464638,0.378172,0.484464,0.391136,0.546176,0.571302
cg00000108,0.951474,0.952636,0.951960,0.942636,0.963885,0.960909,0.935152,0.961977,0.933759,0.938655,...,0.961619,0.961346,0.964522,0.944376,0.936806,0.913477,0.949904,0.951558,0.954932,0.922968
cg00000109,0.899507,0.896365,0.858829,0.844626,0.797604,0.841048,0.865766,0.871188,0.895762,0.913243,...,0.834972,0.883837,0.878783,0.808218,0.866698,0.851090,0.830295,0.888915,0.869469,0.794161
cg00000165,0.162161,0.140993,0.179489,0.183277,0.182863,0.217246,0.217256,0.236196,0.157192,0.197825,...,0.134914,0.185716,0.194467,0.155407,0.182223,0.471573,0.126177,0.153053,0.167795,0.152402
cg00000236,0.783451,0.719850,0.743973,0.735974,0.790358,0.827679,0.718317,0.772257,0.732836,0.781257,...,0.759885,0.738814,0.751406,0.840779,0.782726,0.740348,0.699439,0.675652,0.716671,0.719669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rs9363764,0.884993,0.575390,0.033317,0.575411,0.036038,0.547606,0.039580,0.042940,0.584236,0.550538,...,0.047767,0.582720,0.910069,0.581646,0.943216,0.089709,0.923271,0.935945,0.546927,0.030692
rs939290,0.947329,0.947207,0.525745,0.496984,0.030874,0.506133,0.954376,0.951915,0.022362,0.500149,...,0.956040,0.955622,0.505769,0.954314,0.533835,0.949370,0.516095,0.511649,0.943399,0.677327
rs951295,0.946336,0.521053,0.525385,0.955580,0.520154,0.050719,0.512903,0.510637,0.525594,0.946099,...,0.509571,0.050995,0.506762,0.956640,0.947660,0.048015,0.053336,0.509812,0.512275,0.050468


In [19]:
requires_gene_mapping = True

if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    {gene_row_ids}
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')

gene_annotation.columns.head()

MemoryError: 

In [ ]:
if requires_gene_mapping:
    identifier_key = 'ID'
    gene_symbol_key = 'Gene Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [ ]:
genetic_data = normalize_gene_symbols_in_index(genetic_data)

genetic_data

In [ ]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

merged_data

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

In [ ]:
if is_available:
    save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')
else:
    save_cohort_info(cohort, JSON_PATH, is_available)
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

In [23]:
# Finished
cohort = accession_num = "GSE78160"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

from utils import *
import io
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

clinical_data

!Series_title	"Identification of a serum 48-lncRNA signature as diagnostic marker for hepatocellular carcinoma and liver cirrhosis"
!Series_summary	"In cancer management, early and accurate diagnosis of hepatocellular carcinoma (HCC) is important for enhancing survival rate of patients. Currently, serum alpha-fetoprotein (AFP) is the only one biomarker for detection of HCC. However, serum AFP is not satisfactory for diagnosis of HCC due to its low accuracy (about 60-70%). In this study, we collected 109 serum samples (discovery set) from healthy control (HC) and patients with chronic hepatitis B (CHB), liver cirrhosis (LC) and HCC, and analyzed them with custom lncRNA microarray. Profiling analysis shows 181 differentially expressed lncRNAs between HCs and patients with CHB, LC and HCC. Then a 48-lncRNA diagnostic signature was identified with 100% predictive accuracy for all subjects in the discovery set. This diagnostic signature was verified with a cross-validation analysis in the d

,!Sample_geo_accession,GSM2068378,GSM2068379,GSM2068380,GSM2068381,GSM2068382,GSM2068383,GSM2068384,GSM2068385,GSM2068386,...,GSM2068543,GSM2068544,GSM2068545,GSM2068546,GSM2068547,GSM2068548,GSM2068549,GSM2068550,GSM2068551,GSM2068552
0,!Sample_characteristics_ch1,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,...,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum,tissue: serum
1,!Sample_characteristics_ch1,disease state: healthy control,disease state: healthy control,disease state: healthy control,disease state: healthy control,disease state: healthy control,disease state: healthy control,disease state: healthy control,disease state: healthy control,disease state: healthy control,...,disease state: hepatocellular carcinoma,disease state: hepatocellular carcinoma,disease state: hepatocellular carcinoma,disease state: hepatocellular carcinoma,disease state: hepatocellular carcinoma,disease state: hepatocellular carcinoma,disease state: hepatocellular carcinoma,disease state: hepatocellular carcinoma,disease state: hepatocellular carcinoma,disease state: hepatocellular carcinoma
2,!Sample_characteristics_ch1,data set: discovery,data set: discovery,data set: discovery,data set: discovery,data set: discovery,data set: discovery,data set: discovery,data set: discovery,data set: discovery,...,data set: validation,data set: validation,data set: validation,data set: validation,data set: validation,data set: validation,data set: validation,data set: validation,data set: validation,data set: validation


In [24]:
tumor_stage_row = clinical_data.iloc[1]
tumor_stage_row.unique()

array(['!Sample_characteristics_ch1', 'disease state: healthy control',
       'disease state: chronic hepatitis B',
       'disease state: liver cirrhosis',
       'disease state: hepatocellular carcinoma'], dtype=object)

In [25]:
is_gene_availabe = True
trait_row = 1
age_row = None
gender_row = None

trait_type = 'binary'

# Verify and use the functions generated by GPT

# 这个函数将组织类型（tissue type）转换为有关癫痫存在与否的二进制值。
# 它是基于特定的假设，即如果组织类型是“胰腺导管腺癌”（Pancreatic Ductal Adenocarcinoma），则认为癫痫存在（返回1）；否则，认为癫痫不存在（返回0）。
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'disease state: chronic hepatitis B':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

In [26]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

c:\Users\legion\Desktop\TAIS_2024SP\TAIS_2024SP\Jiayi\..\utils.py:218: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)


,GSM2068378,GSM2068379,GSM2068380,GSM2068381,GSM2068382,GSM2068383,GSM2068384,GSM2068385,GSM2068386,GSM2068387,...,GSM2068543,GSM2068544,GSM2068545,GSM2068546,GSM2068547,GSM2068548,GSM2068549,GSM2068550,GSM2068551,GSM2068552
Hepatitis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
genetic_data = get_genetic_data(matrix_file)
genetic_data

,GSM2068378,GSM2068379,GSM2068380,GSM2068381,GSM2068382,GSM2068383,GSM2068384,GSM2068385,GSM2068386,GSM2068387,...,GSM2068543,GSM2068544,GSM2068545,GSM2068546,GSM2068547,GSM2068548,GSM2068549,GSM2068550,GSM2068551,GSM2068552
ID,,,,,,,,,,,,,,,,,,,,,
1,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,...,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856
2,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,...,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856
3,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,...,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856
4,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,...,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,12.123889,8.643856
5,8.643856,10.890497,9.298094,10.515535,10.303984,8.643856,8.643856,8.643856,8.643856,8.643856,...,12.128642,8.643856,8.643856,8.643856,10.474615,8.643856,8.643856,8.643856,8.643856,8.643856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,14.147935,14.201631,13.710442,14.055458,13.706570,12.381570,14.072761,12.585021,13.918598,14.086638,...,15.719566,13.831713,14.923421,14.254049,14.505217,12.853661,13.391889,15.424096,14.041115,14.297063
2517,14.293053,15.253498,14.591874,15.000236,15.315987,15.531363,15.179144,14.094255,15.180399,15.388774,...,14.262666,12.777848,13.929949,13.053648,13.675200,13.590098,14.167543,15.472563,14.227284,14.443754
2518,14.651719,10.878482,12.772778,12.068672,12.434913,13.155134,12.168948,11.266066,8.643856,12.914463,...,14.380008,12.462397,13.909031,13.137195,13.509931,14.440875,15.480936,15.092393,13.922251,13.806244


In [28]:
requires_gene_mapping = True

if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    {gene_row_ids}
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')

gene_annotation.columns

{'ID': ['1', '2', '3', '4', '5'], 'SPOT_ID': ['UBC', 'NEIF3D', 'GRM4', 'GAPDH', 'NC-1'], 'SEQUENCE': ['TCGAGCCCAGTGACACCATCGAGAACGTCAAAGCAAAGATCCAGGACAAGGAAGG', 'ACTACGACAAAGCCTTTGACCGCATCACCACGAGGAGTGAGAAGCCACTGCGGAGCAT', 'ATCCTCTTTGTCCTCAGCTCCTCCTGCTTTCTTGGGTCCCACCAGTGTCACTTTTCTGCC', 'AGTGGTGGACCTGACCTGCCGTCTAGAAAAACCTGCCAAATATGATGACATCAAGAAG', 'TCAGAGTAGACTCGACTGCGATGTACTCAGCACAGCAACGTCGTAGGACAGTCGATCTTG']}

    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this f

Index(['ID', 'SPOT_ID', 'SEQUENCE'], dtype='object')

In [30]:
if requires_gene_mapping:
    identifier_key = 'ID'
    gene_symbol_key = 'SPOT_ID'
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [31]:
genetic_data = normalize_gene_symbols_in_index(genetic_data)

genetic_data

,GSM2068378,GSM2068379,GSM2068380,GSM2068381,GSM2068382,GSM2068383,GSM2068384,GSM2068385,GSM2068386,GSM2068387,...,GSM2068543,GSM2068544,GSM2068545,GSM2068546,GSM2068547,GSM2068548,GSM2068549,GSM2068550,GSM2068551,GSM2068552
7SK,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,...,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856
AIR,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,...,11.372260,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856
COPG2IT1,9.603117,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,...,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856
DGCR5,8.643856,11.847305,10.266793,11.977783,11.383699,10.626553,10.721836,9.398005,11.262412,11.078104,...,12.946249,8.952611,10.381489,10.757499,8.643856,9.477524,11.104535,12.809209,11.095331,8.643856
DISC2,11.094534,9.750400,8.643856,10.825720,10.434356,8.643856,10.771369,9.763246,8.643856,10.355665,...,12.394881,8.643856,11.579984,9.680585,10.862449,9.290551,8.643856,10.758394,10.532171,8.643856
DIO3OS,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,...,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.959334,8.643856,8.643856
EMX2OS,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,...,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856
GAPDH,8.643856,8.929325,8.686277,9.094759,8.915263,8.643856,8.643856,8.643856,8.643856,8.643856,...,9.856177,8.643856,9.783056,8.770832,9.167772,8.643856,9.033588,8.643856,10.304445,10.007780
GAS5,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,...,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856
GRM4,8.643856,8.938785,8.643856,9.057079,8.643856,8.643856,8.751193,8.643856,8.643856,8.762620,...,9.739373,8.751991,9.431670,8.738022,8.869630,8.643856,9.516300,8.655169,11.837607,11.610034


In [34]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

merged_data

,Hepatitis,7SK,AIR,COPG2IT1,DGCR5,DISC2,DIO3OS,EMX2OS,GAPDH,GAS5,...,SNHG5,SNHG6,SPRY4-IT1,TUG1,UBC,UCA1,WT1-AS,XIST,ZFAS1,MASCRNA
GSM2068378,0.0,8.643856,8.643856,9.603117,8.643856,11.094534,8.643856,8.643856,8.643856,8.643856,...,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856
GSM2068379,0.0,8.643856,8.643856,8.643856,11.847305,9.750400,8.643856,8.643856,8.929325,8.643856,...,8.643856,8.643856,8.643856,11.537742,8.643856,8.643856,8.643856,10.070961,8.643856,8.643856
GSM2068380,0.0,8.643856,8.643856,8.643856,10.266793,8.643856,8.643856,8.643856,8.686277,8.643856,...,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856
GSM2068381,0.0,8.643856,8.643856,8.643856,11.977783,10.825720,8.643856,8.643856,9.094759,8.643856,...,8.643856,8.643856,8.643856,11.584173,8.643856,8.643856,8.643856,8.807389,8.643856,8.643856
GSM2068382,0.0,8.643856,8.643856,8.643856,11.383699,10.434356,8.643856,8.643856,8.915263,8.643856,...,8.643856,8.643856,8.643856,8.892180,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM2068548,0.0,8.643856,8.643856,8.643856,9.477524,9.290551,8.643856,8.643856,8.643856,8.643856,...,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856
GSM2068549,0.0,8.643856,8.643856,8.643856,11.104535,8.643856,8.643856,8.643856,9.033588,8.643856,...,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856
GSM2068550,0.0,8.643856,8.643856,8.643856,12.809209,10.758394,8.959334,8.643856,8.643856,8.643856,...,8.643856,8.643856,8.643856,11.150089,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856
GSM2068551,0.0,8.643856,8.643856,8.643856,11.095331,10.532171,8.643856,8.643856,10.304445,8.643856,...,8.643856,8.643856,8.643856,10.231673,8.643856,8.643856,8.643856,8.643856,8.643856,8.643856


In [35]:
print(f"The merged dataset contains {len(merged_data)} samples.")
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

The merged dataset contains 175 samples.
For the feature 'Hepatitis', the least common label is '1.0' with 45 occurrences. This represents 25.71% of the dataset.
The distribution of the feature 'Hepatitis' in this dataset is fine.



False

In [36]:
if is_available:
    save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')
else:
    save_cohort_info(cohort, JSON_PATH, is_available)
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

A new JSON file was created at: /Users/legion/Desktop/Courses/IS389/output\Jiayi\Hepatitis\cohort_info.json


In [21]:
# Stopped: No obvious trait convert
cohort = accession_num = "GSE85550"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

from utils import *
import io
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

clinical_data

!Series_title	"Molecular signature predictive of long-term liver fibrosis progression to inform anti-fibrotic drug development"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"


,!Sample_geo_accession,GSM4557370,GSM4557371,GSM4557372,GSM4557373,GSM4557374,GSM4557375,GSM4557376,GSM4557377,GSM4557378,...,GSM5517662,GSM5517663,GSM5517664,GSM5517665,GSM5517666,GSM5517667,GSM5517668,GSM5517669,GSM5517670,GSM5517671
0,!Sample_characteristics_ch1,fibrosis stage: 0,fibrosis stage: 0,fibrosis stage: 0,fibrosis stage: 0,fibrosis stage: 0,fibrosis stage: 0,fibrosis stage: 0,fibrosis stage: 0,fibrosis stage: 0,...,tissue: Liver,tissue: Liver,tissue: Liver,tissue: Liver,tissue: Liver,tissue: Liver,tissue: Liver,tissue: Liver,tissue: Liver,tissue: Liver
1,!Sample_characteristics_ch1,pls risk prediction: High,pls risk prediction: Intermediate,pls risk prediction: Intermediate,pls risk prediction: High,pls risk prediction: Intermediate,pls risk prediction: High,pls risk prediction: High,pls risk prediction: Intermediate,pls risk prediction: High,...,sample group: Vehicle control,sample group: Vehicle control,sample group: Vehicle control,sample group: Vehicle control,sample group: Vehicle control,sample group: Vehicle control,sample group: Vehicle control,sample group: Vehicle control,sample group: Vehicle control,sample group: Vehicle control
2,!Sample_characteristics_ch1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,compound: DMSO,compound: DMSO,compound: DMSO,compound: DMSO,compound: DMSO,compound: DMSO,compound: DMSO,compound: DMSO,compound: DMSO,compound: DMSO
3,!Sample_characteristics_ch1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,concentration: 0.1%,concentration: 0.1%,concentration: 0.1%,concentration: 0.1%,concentration: 0.1%,concentration: 0.1%,concentration: 0.1%,concentration: 0.1%,concentration: 0.1%,concentration: 0.1%


In [8]:
cohort = accession_num = "GSE113392"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/zhangmianchen/Downloads/DATA/GEO/Hepatitis/GSE113392/GSE113392_family.soft.gz',
 '/Users/zhangmianchen/Downloads/DATA/GEO/Hepatitis/GSE113392/GSE113392_series_matrix.txt.gz')

### Initial filtering and clinical data preprocessing

In [9]:
import gzip

In [10]:
def line_generator(source, source_type):
    """Generator that yields lines from a file or a string.

    Parameters:
    - source: File path or string content.
    - source_type: 'file' or 'string'.
    """
    if source_type == 'file':
        with gzip.open(source, 'rt') as f:
            for line in f:
                yield line.strip()
    elif source_type == 'string':
        for line in source.split('\n'):
            yield line.strip()
    else:
        raise ValueError("source_type must be 'file' or 'string'")

In [11]:
from typing import Callable, Optional, List, Tuple, Union, Any
import pandas as pd

In [12]:
def filter_content_by_prefix(
    source: str,
    prefixes_a: List[str],
    prefixes_b: Optional[List[str]] = None,
    unselect: bool = False,
    source_type: str = 'file',
    return_df_a: bool = True,
    return_df_b: bool = True
) -> Tuple[Union[str, pd.DataFrame], Optional[Union[str, pd.DataFrame]]]:
    """
    Filters rows from a file or a list of strings based on specified prefixes.

    Parameters:
    - source (str): File path or string content to filter.
    - prefixes_a (List[str]): Primary list of prefixes to filter by.
    - prefixes_b (Optional[List[str]]): Optional secondary list of prefixes to filter by.
    - unselect (bool): If True, selects rows that do not start with the specified prefixes.
    - source_type (str): 'file' if source is a file path, 'string' if source is a string of text.
    - return_df_a (bool): If True, returns filtered content for prefixes_a as a pandas DataFrame.
    - return_df_b (bool): If True, and if prefixes_b is provided, returns filtered content for prefixes_b as a pandas DataFrame.

    Returns:
    - Tuple: A tuple where the first element is the filtered content for prefixes_a, and the second element is the filtered content for prefixes_b.
    """
    filtered_lines_a = []
    filtered_lines_b = []
    prefix_set_a = set(prefixes_a)
    if prefixes_b is not None:
        prefix_set_b = set(prefixes_b)

    # Use generator to get lines
    for line in line_generator(source, source_type):
        matched_a = any(line.startswith(prefix) for prefix in prefix_set_a)
        if matched_a != unselect:
            filtered_lines_a.append(line)
        if prefixes_b is not None:
            matched_b = any(line.startswith(prefix) for prefix in prefix_set_b)
            if matched_b != unselect:
                filtered_lines_b.append(line)

    filtered_content_a = '\n'.join(filtered_lines_a)
    if return_df_a:
        filtered_content_a = pd.read_csv(io.StringIO(filtered_content_a), delimiter='\t', low_memory=False, on_bad_lines='skip')
    filtered_content_b = None
    if filtered_lines_b:
        filtered_content_b = '\n'.join(filtered_lines_b)
        if return_df_b:
            filtered_content_b = pd.read_csv(io.StringIO(filtered_content_b), delimiter='\t', low_memory=False, on_bad_lines='skip')

    return filtered_content_a, filtered_content_b



In [13]:
def get_background_and_clinical_data(file_path,
                                     prefixes_a=['!Series_title', '!Series_summary', '!Series_overall_design'],
                                     prefixes_b=['!Sample_geo_accession', '!Sample_characteristics_ch1']):
    """Extract from a matrix file the background information about the dataset, and sample characteristics data"""
    background_info, clinical_data = filter_content_by_prefix(file_path, prefixes_a, prefixes_b, unselect=False,
                                                              source_type='file',
                                                              return_df_a=False, return_df_b=True)
    return background_info, clinical_data

In [14]:
import io
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Loci-specific differences in blood DNA methylation in HBV-negative populations at risk for hepatocellular carcinoma development - pre-diagnostic HCC blood samples"
!Series_summary	"Genome wide DNA methylation profiling of blood samples collected from patients prior to diagnosis with hepatocellular carcinoma (HCC) vs. blood samples collected from healthy individuals without family history of cancer. The Illumina Infinium 450K Human DNA methylation Beadchip v1.2 was used to obtain DNA methylation profiles across approximately 450,000 CpGs in human samples corresponding to cases (pre-diagnostic HCC) and controls. Samples included 21 cases and 21 controls. Cases were matched with controls on gender, age, ethnicity, hepatitis C infection, and diabetes. The presence or absence of HCC in our study was determined based on the AASLD criteria."
!Series_overall_design	"Bisulphite converted DNA from the 42 samples were hybridised to the Illumina Infinium 450k Human Methylation Beadc

In [15]:
clinical_data.head()

,!Sample_geo_accession,GSM3104805,GSM3104806,GSM3104807,GSM3104808,GSM3104809,GSM3104810,GSM3104811,GSM3104812,GSM3104813,...,GSM3104837,GSM3104838,GSM3104839,GSM3104840,GSM3104841,GSM3104842,GSM3104843,GSM3104844,GSM3104845,GSM3104846
0,!Sample_characteristics_ch1,case/control: Control - Healthy,case/control: Case - pre-diagnostic hepatocell...,case/control: Control - Healthy,case/control: Case - pre-diagnostic hepatocell...,case/control: Control - Healthy,case/control: Case - pre-diagnostic hepatocell...,case/control: Control - Healthy,case/control: Case - pre-diagnostic hepatocell...,case/control: Control - Healthy,...,case/control: Control - Healthy,case/control: Case - pre-diagnostic hepatocell...,case/control: Control - Healthy,case/control: Case - pre-diagnostic hepatocell...,case/control: Control - Healthy,case/control: Case - pre-diagnostic hepatocell...,case/control: Control - Healthy,case/control: Case - pre-diagnostic hepatocell...,case/control: Control - Healthy,case/control: Case - pre-diagnostic hepatocell...
1,!Sample_characteristics_ch1,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,...,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood,sample type: human whole blood
2,!Sample_characteristics_ch1,disease state: normal,disease state: hepatocellular carcinoma (HCC),disease state: normal,disease state: hepatocellular carcinoma (HCC),disease state: normal,disease state: hepatocellular carcinoma (HCC),disease state: normal,disease state: hepatocellular carcinoma (HCC),disease state: normal,...,disease state: normal,disease state: hepatocellular carcinoma (HCC),disease state: normal,disease state: hepatocellular carcinoma (HCC),disease state: normal,disease state: hepatocellular carcinoma (HCC),disease state: normal,disease state: hepatocellular carcinoma (HCC),disease state: normal,disease state: hepatocellular carcinoma (HCC)


In [16]:
def get_unique_values_by_row(dataframe, max_len=30):
    """
    Organize the unique values in each row of the given dataframe, to get a dictionary
    :param dataframe:
    :param max_len:
    :return:
    """
    if '!Sample_geo_accession' in dataframe.columns:
        dataframe = dataframe.drop(columns=['!Sample_geo_accession'])
    unique_values_dict = {}
    for index, row in dataframe.iterrows():
        unique_values = list(row.unique())[:max_len]
        unique_values_dict[index] = unique_values
    return unique_values_dict

In [17]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['case/control: Control - Healthy',
  'case/control: Case - pre-diagnostic hepatocellular carcinoma (HCC)'],
 1: ['sample type: human whole blood'],
 2: ['disease state: normal', 'disease state: hepatocellular carcinoma (HCC)']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [18]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Hepatitis\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Hepatitis\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an intege

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [19]:
age_row = gender_row = None
convert_age = convert_gender = None

In [20]:
is_gene_availabe = True
trait_row = 0
age_row = None
gender_row = None
trait_type = 'binary'

In [21]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [22]:
# Verify and use the functions generated by GPT

def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'case/control: Case - pre-diagnostic hepatocellular carcinoma (HCC)':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present


def convert_age(age_string):
    """
    Convert age string to a continuous numerical value.
    Unknown values are converted to None.
    """
    if age_string.lower() == 'n.a.':
        return None
    try:
        # Extract age as an integer from the string
        age = int(age_string.split(': ')[1])
        return age
    except (ValueError, IndexError):
        # In case of any format error or unexpected string structure
        return None


# It sometimes maps 'female' to 0, and sometimes 1. Does it matter?
def convert_gender(gender_string):
    """
    Convert gender string to a binary value.
    'female' is represented as 1, 'male' as 0.
    Unknown values are converted to None.
    """
    if gender_string.lower() == 'male: 0':
        return 0
    elif gender_string.lower() == 'male: 1':
        return 1
    else:
        return None

In [23]:
def get_feature_data(clinical_df, row_id, feature, convert_fn):
    """select the row corresponding to a feature in the sample characteristics dataframe, and convert the feature into
    a binary or continuous variable"""
    clinical_df = clinical_df.iloc[row_id:row_id + 1].drop(columns=['!Sample_geo_accession'], errors='ignore')
    clinical_df.index = [feature]
    clinical_df = clinical_df.applymap(convert_fn)

    return clinical_df

In [24]:
def geo_select_clinical_features(clinical_df: pd.DataFrame, trait: str, trait_row: int,
                                 convert_trait: Callable,
                                 age_row: Optional[int] = None,
                                 convert_age: Optional[Callable] = None,
                                 gender_row: Optional[int] = None,
                                 convert_gender: Optional[Callable] = None) -> pd.DataFrame:
    """
    Extracts and processes specific clinical features from a DataFrame representing
    sample characteristics in the GEO database series.

    Parameters:
    - clinical_df (pd.DataFrame): DataFrame containing clinical data.
    - trait (str): The trait of interest.
    - trait_row (int): Row identifier for the trait in the DataFrame.
    - convert_trait (Callable): Function to convert trait data into a desired format.
    - age_row (int, optional): Row identifier for age data. Default is None.
    - convert_age (Callable, optional): Function to convert age data. Default is None.
    - gender_row (int, optional): Row identifier for gender data. Default is None.
    - convert_gender (Callable, optional): Function to convert gender data. Default is None.

    Returns:
    pd.DataFrame: A DataFrame containing the selected and processed clinical features.
    """
    feature_list = []

    trait_data = get_feature_data(clinical_df, trait_row, trait, convert_trait)
    feature_list.append(trait_data)
    if age_row is not None:
        age_data = get_feature_data(clinical_df, age_row, 'Age', convert_age)
        feature_list.append(age_data)
    if gender_row is not None:
        gender_data = get_feature_data(clinical_df, gender_row, 'Gender', convert_gender)
        feature_list.append(gender_data)

    selected_clinical_df = pd.concat(feature_list, axis=0)
    return selected_clinical_df

In [25]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM3104805,GSM3104806,GSM3104807,GSM3104808,GSM3104809,GSM3104810,GSM3104811,GSM3104812,GSM3104813,GSM3104814,...,GSM3104837,GSM3104838,GSM3104839,GSM3104840,GSM3104841,GSM3104842,GSM3104843,GSM3104844,GSM3104845,GSM3104846
Hepatitis,0,1,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,0,1


### Genetic data preprocessing and final filtering

In [26]:
def get_genetic_data(file_path):
    """Read the gene expression data into a dataframe, and adjust its format"""
    genetic_data = pd.read_csv(file_path, compression='gzip', skiprows=52, comment='!', delimiter='\t')
    genetic_data = genetic_data.dropna()
    genetic_data = genetic_data.rename(columns={'ID_REF': 'ID'}).astype({'ID': 'str'})
    genetic_data.set_index('ID', inplace=True)

    return genetic_data


In [27]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM3104805,GSM3104806,GSM3104807,GSM3104808,GSM3104809,GSM3104810,GSM3104811,GSM3104812,GSM3104813,GSM3104814,...,GSM3104837,GSM3104838,GSM3104839,GSM3104840,GSM3104841,GSM3104842,GSM3104843,GSM3104844,GSM3104845,GSM3104846
ID,,,,,,,,,,,,,,,,,,,,,
cg00000029,0.641753,0.536023,0.480923,0.413408,0.503781,0.484459,0.322564,0.598879,0.472014,0.492969,...,0.469127,0.473445,0.488830,0.434073,0.543320,0.533835,0.532300,0.554394,0.654268,0.534820
cg00000108,0.946797,0.964972,0.949461,0.941012,0.917665,0.917262,0.945542,0.949976,0.922315,0.953664,...,0.953520,0.942573,0.935418,0.947862,0.955645,0.942716,0.942501,0.948831,0.954096,0.912201
cg00000109,0.898792,0.904099,0.908054,0.890208,0.883757,0.861751,0.897760,0.881591,0.899898,0.913731,...,0.915518,0.880797,0.911363,0.786655,0.865565,0.890660,0.898855,0.890524,0.836090,0.878666
cg00000165,0.244225,0.203119,0.276537,0.140747,0.124542,0.132241,0.149900,0.146812,0.208076,0.171069,...,0.232690,0.210047,0.209973,0.155052,0.168326,0.162929,0.156828,0.211841,0.196233,0.136576
cg00000236,0.776429,0.809852,0.809300,0.767984,0.737150,0.728721,0.732519,0.869120,0.806040,0.749944,...,0.730795,0.833923,0.771805,0.707987,0.824090,0.767424,0.734655,0.814077,0.786997,0.774174


In [28]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['cg00000029',
 'cg00000108',
 'cg00000109',
 'cg00000165',
 'cg00000236',
 'cg00000289',
 'cg00000292',
 'cg00000321',
 'cg00000363',
 'cg00000622',
 'cg00000658',
 'cg00000714',
 'cg00000721',
 'cg00000734',
 'cg00000769',
 'cg00000807',
 'cg00000884',
 'cg00000905',
 'cg00000924',
 'cg00000948']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [29]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['cg00000029', 'cg00000108', 'cg00000109', 'cg00000165', 'cg00000236', 'cg00000289', 'cg00000292', 'cg00000321', 'cg00000363', 'cg00000622', 'cg00000658', 'cg00000714', 'cg00000721', 'cg00000734', 'cg00000769', 'cg00000807', 'cg00000884', 'cg00000905', 'cg00000924', 'cg00000948']\n"


If not required, jump directly to the gene normalization step

In [30]:
requires_gene_mapping = False

In [31]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [32]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    {gene_row_ids}
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')

In [33]:
if requires_gene_mapping:
    identifier_key = 'ID'
    gene_symbol_key = 'UCSC_RefGene_Name'
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [34]:
def normalize_gene_symbols_in_index(gene_df):
    """Normalize the human gene symbols at the index of a dataframe, and replace the index with its normalized version.
    Remove the rows where the index failed to be normalized."""
    normalized_gene_list = normalize_gene_symbols(gene_df.index.tolist())
    assert len(normalized_gene_list) == len(gene_df.index)
    gene_df.index = normalized_gene_list
    gene_df = gene_df[gene_df.index.notnull()]
    return gene_df

In [35]:
def normalize_gene_symbols(gene_symbols, batch_size=1000):
    """Normalize human gene symbols in batches using the 'mygenes' library"""
    mg = mygene.MyGeneInfo()
    normalized_genes = {}

    # Process in batches
    for i in range(0, len(gene_symbols), batch_size):
        batch = gene_symbols[i:i + batch_size]
        results = mg.querymany(batch, scopes='symbol', fields='symbol', species='human')

        # Update the normalized_genes dictionary with results from this batch
        for gene in results:
            normalized_genes[gene['query']] = gene.get('symbol', None)

    # Return the normalized symbols in the same order as the input
    return [normalized_genes.get(symbol) for symbol in gene_symbols]

In [36]:
import mygene

In [37]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

1000 input query terms found no hit:	['cg00000029', 'cg00000108', 'cg00000109', 'cg00000165', 'cg00000236', 'cg00000289', 'cg00000292', '
1000 input query terms found no hit:	['cg00040862', 'cg00040873', 'cg00040923', 'cg00040945', 'cg00040986', 'cg00041014', 'cg00041047', '
1000 input query terms found no hit:	['cg00087971', 'cg00088007', 'cg00088026', 'cg00088033', 'cg00088042', 'cg00088079', 'cg00088080', '
1000 input query terms found no hit:	['cg00134190', 'cg00134210', 'cg00134212', 'cg00134295', 'cg00134340', 'cg00134415', 'cg00134539', '
1000 input query terms found no hit:	['cg00182202', 'cg00182273', 'cg00182288', 'cg00182294', 'cg00182416', 'cg00182421', 'cg00182534', '
1000 input query terms found no hit:	['cg00237631', 'cg00237752', 'cg00237825', 'cg00237876', 'cg00237904', 'cg00238038', 'cg00238052', '
1000 input query terms found no hit:	['cg00290039', 'cg00290086', 'cg00290101', 'cg00290158', 'cg00290218', 'cg00290303', 'cg00290340', '
1000 input query terms found no hi

In [38]:
def geo_merge_clinical_genetic_data(clinical_df, genetic_df):
    """
    Merge the clinical features and gene expression features from two dataframes into one dataframe
    """
    if 'ID' in genetic_df.columns:
        genetic_df = genetic_df.rename(columns={'ID': 'Gene'})
    if 'Gene' in genetic_df.columns:
        genetic_df = genetic_df.set_index('Gene')
    merged_data = pd.concat([clinical_df, genetic_df], axis=0).T.dropna()
    return merged_data


In [39]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [40]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 42 samples.


In [41]:
def judge_and_remove_biased_features(df, trait, trait_type):
    assert trait_type in ["binary", "continuous"], f"The trait must be either a binary or a continuous variable!"
    if trait_type == "binary":
        trait_biased = judge_binary_variable_biased(df, trait)
    else:
        trait_biased = judge_continuous_variable_biased(df, trait)
    if trait_biased:
        print(f"The distribution of the feature \'{trait}\' in this dataset is severely biased.\n")
    else:
        print(f"The distribution of the feature \'{trait}\' in this dataset is fine.\n")
    if "Age" in df.columns:
        age_biased = judge_continuous_variable_biased(df, 'Age')
        if age_biased:
            print(f"The distribution of the feature \'Age\' in this dataset is severely biased.\n")
            df = df.drop(columns='Age')
        else:
            print(f"The distribution of the feature \'Age\' in this dataset is fine.\n")
    if "Gender" in df.columns:
        gender_biased = judge_binary_variable_biased(df, 'Gender')
        if gender_biased:
            print(f"The distribution of the feature \'Gender\' in this dataset is severely biased.\n")
            df = df.drop(columns='Gender')
        else:
            print(f"The distribution of the feature \'Gender\' in this dataset is fine.\n")

    return trait_biased, df


In [42]:
def judge_binary_variable_biased(dataframe, col_name, min_proportion=0.1, min_num=5):
    """
    Check if the distribution of a binary variable in the dataset is too biased to be usable for analysis
    :param dataframe:
    :param col_name:
    :param min_proportion:
    :param min_num:
    :return:
    """
    label_counter = dataframe[col_name].value_counts()
    total_samples = len(dataframe)
    rare_label_num = label_counter.min()
    rare_label = label_counter.idxmin()
    rare_label_proportion = rare_label_num / total_samples

    print(
        f"For the feature \'{col_name}\', the least common label is '{rare_label}' with {rare_label_num} occurrences. This represents {rare_label_proportion:.2%} of the dataset.")

    biased = (len(label_counter) < 2) or ((rare_label_proportion < min_proportion) and (rare_label_num < min_num))
    return bool(biased)

In [43]:
def judge_continuous_variable_biased(dataframe, col_name):
    """Check if the distribution of a continuous variable in the dataset is too biased to be usable for analysis.
    As a starting point, we consider it biased if all values are the same. For the next step, maybe ask GPT to judge
    based on quartile statistics combined with its common sense knowledge about this feature.
    """
    quartiles = dataframe[col_name].quantile([0.25, 0.5, 0.75])
    min_value = dataframe[col_name].min()
    max_value = dataframe[col_name].max()

    # Printing quartile information
    print(f"Quartiles for '{col_name}':")
    print(f"  25%: {quartiles[0.25]}")
    print(f"  50% (Median): {quartiles[0.5]}")
    print(f"  75%: {quartiles[0.75]}")
    print(f"Min: {min_value}")
    print(f"Max: {max_value}")

    biased = min_value == max_value

    return bool(biased)

In [44]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Hepatitis', the least common label is '0.0' with 21 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Hepatitis' in this dataset is fine.



False

In [45]:
def save_cohort_info(cohort: str, info_path: str, is_available: bool, is_biased: Optional[bool] = None,
                     df: Optional[pd.DataFrame] = None, note: str = '') -> None:
    """
    Add or update information about the usability and quality of a dataset for statistical analysis.

    Parameters:
    cohort (str): A unique identifier for the dataset.
    info_path (str): File path to the JSON file where records are stored.
    is_available (bool): Indicates whether both the genetic data and trait data are available in the dataset, and can be
     preprocessed into a dataframe.
    is_biased (bool, optional): Indicates whether the dataset is too biased to be usable.
        Required if `is_available` is True.
    df (pandas.DataFrame, optional): The preprocessed dataset. Required if `is_available` is True.
    note (str, optional): Additional notes about the dataset.

    Returns:
    None: The function does not return a value but updates or creates a record in the specified JSON file.
    """
    if is_available:
        assert (df is not None) and (is_biased is not None), "'df' and 'is_biased' should be provided if this cohort " \
                                                             "is relevant."
    is_usable = is_available and (not is_biased)
    new_record = {"is_usable": is_usable,
                  "is_available": is_available,
                  "is_biased": is_biased if is_available else None,
                  "has_age": "Age" in df.columns if is_available else None,
                  "has_gender": "Gender" in df.columns if is_available else None,
                  "sample_size": len(df) if is_available else None,
                  "note": note}
    
    if not os.path.exists(info_path):
        with open(info_path, 'w') as file:
            json.dump({}, file)
        print(f"A new JSON file was created at: {info_path}")

    with open(info_path, "r") as file:
        records = json.load(file)
    records[cohort] = new_record

    temp_path = info_path + ".tmp"
    try:
        with open(temp_path, 'w') as file:
            json.dump(records, file)
        os.replace(temp_path, info_path)

    except Exception as e:
        print(f"An error occurred: {e}")
        if os.path.exists(temp_path):
            os.remove(temp_path)
        raise

In [46]:
import json
if is_available:
    save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')
else:
    save_cohort_info(cohort, JSON_PATH, is_available)

In [47]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

### 3. Do regression & Cross Validation

In [48]:
def read_json_to_dataframe(json_file: str) -> pd.DataFrame:
    """
    Reads a JSON file and converts it into a pandas DataFrame.

    Args:
    json_file (str): The path to the JSON file containing the data.

    Returns:
    DataFrame: A pandas DataFrame with the JSON data.
    """
    with open(json_file, 'r') as file:
        data = json.load(file)
    return pd.DataFrame.from_dict(data, orient='index').reset_index().rename(columns={'index': 'cohort_id'})

In [49]:
def filter_and_rank_cohorts(json_file: str, condition: Union[str, None] = None) -> Tuple[
    Union[str, None], pd.DataFrame]:
    """
    Reads a JSON file, filters cohorts based on usability and an optional condition, then ranks them by sample size.

    Args:
    json_file (str): The path to the JSON file containing the data.
    condition (str, optional): An additional condition for filtering. If None, only 'is_usable' is considered.

    Returns:
    Tuple: A tuple containing the best cohort ID (str or None if no suitable cohort is found) and
           the filtered and ranked DataFrame.
    """
    # Read the JSON file into a DataFrame
    df = read_json_to_dataframe(json_file)

    if condition:
        filtered_df = df[(df['is_usable'] == True) & (df[condition] == True)]
    else:
        filtered_df = df[df['is_usable'] == True]

    ranked_df = filtered_df.sort_values(by='sample_size', ascending=False)
    best_cohort_id = ranked_df.iloc[0]['cohort_id'] if not ranked_df.empty else None

    return best_cohort_id, ranked_df


In [50]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE113392,True,True,False,False,False,42,


In [51]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

In [52]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note


In [53]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [ ]:
def detect_batch_effect(X):
    """
    Detect potential batch effects in a dataset using eigenvalues of XX^T.

    Args:
    X (numpy.ndarray): A feature matrix with shape (n_samples, n_features).

    Returns:
    bool: True if a potential batch effect is detected, False otherwise.
    """
    n_samples = X.shape[0]

    # Computing XX^T
    XXt = np.dot(X, X.T)

    # Compute the eigenvalues of XX^T
    eigen_values = np.linalg.eigvalsh(XXt)  # Using eigvalsh since XX^T is symmetric
    eigen_values = sorted(eigen_values, reverse=True)

    # Check for large gaps in the eigenvalues
    for i in range(len(eigen_values) - 1):
        gap = eigen_values[i] - eigen_values[i + 1]
        if gap > 1 / n_samples:  # You may need to adjust this threshold
            return True

    return False

In [ ]:
import numpy as np
has_batch_effect = detect_batch_effect(X)
has_batch_effect

In [ ]:
from sparse_lmm import VariableSelection


In [ ]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [ ]:
def cross_validation(X, Y, Z, model_constructor, model_params, k=5, target_type='binary'):
    assert target_type in ['binary', 'continuous'], "The target type must be chosen from 'binary' or 'continuous'"
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    fold_size = len(X) // k
    performances = []

    for i in range(k):
        # Split data into train and test based on the current fold
        test_indices = indices[i * fold_size: (i + 1) * fold_size]
        train_indices = np.setdiff1d(indices, test_indices)

        X_train, X_test = X[train_indices], X[test_indices]
        Y_train, Y_test = Y[train_indices], Y[test_indices]
        Z_train, Z_test = Z[train_indices], Z[test_indices]

        normalized_X_train, normalized_X_test = normalize_data(X_train, X_test)
        normalized_Z_train, normalized_Z_test = normalize_data(Z_train, Z_test)

        # model = model_constructor(**model_params)
        model = ResidualizationRegressor(model_constructor, model_params)
        model.fit(normalized_X_train, Y_train, normalized_Z_train)
        predictions = model.predict(normalized_X_test, normalized_Z_test)

        if target_type == 'binary':
            predictions = (predictions > 0.5).astype(int)
            Y_test = (Y_test > 0.5).astype(int)
            performance = accuracy_score(Y_test, predictions)
        elif target_type == 'continuous':
            performance = mean_squared_error(Y_test, predictions)

        performances.append(performance)

    cv_mean = np.mean(performances)
    cv_std = np.std(performances)

    if target_type == 'binary':
        print(f'The cross-validation accuracy is {(cv_mean * 100):.2f}% ± {(cv_std * 100):.2f}%')
    else:
        print(f'The cross-validation MSE is {(cv_mean * 100):.2f} ± {(cv_std * 100):.2f}')

    return cv_mean, cv_std

In [ ]:
def normalize_data(X_train, X_test=None):
    """Compute the mean and standard deviation statistics of the training data, use them to normalize the training data,
    and optionally the test data"""
    mean = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0)

    # Handling columns with std = 0
    std_no_zero = np.where(std == 0, 1, std)

    # Normalize X_train
    X_train_normalized = (X_train - mean) / std_no_zero
    # Set normalized values to 0 where std was 0
    X_train_normalized[:, std == 0] = 0

    if X_test is not None:
        X_test_normalized = (X_test - mean) / std_no_zero
        X_test_normalized[:, std == 0] = 0
    else:
        X_test_normalized = None

    return X_train_normalized, X_test_normalized

In [ ]:
class ResidualizationRegressor:
    def __init__(self, regression_model_constructor, params=None):
        if params is None:
            params = {}
        self.regression_model = regression_model_constructor(**params)
        self.beta_Z = None  # Coefficients for regression of Y on Z
        self.beta_X = None  # Coefficients for regression of residual on X
        self.neg_log_p_values = None  # Negative logarithm of p-values
        self.p_values = None  # Actual p-values

    def _reshape_data(self, data):
        """
        Reshape the data to ensure it's in the correct format (2D array).

        :param data: The input data (can be 1D or 2D array).
        :return: Reshaped 2D array.
        """
        if data.ndim == 1:
            return data.reshape(-1, 1)
        return data

    def _reshape_output(self, data):
        """
        Reshape the output data to ensure it's in the correct format (1D array).

        :param data: The output data (can be 1D or 2D array).
        :return: Reshaped 1D array.
        """
        if data.ndim == 2 and data.shape[1] == 1:
            return data.ravel()
        return data
    
    def fit(self, X, Y, Z):
        X = self._reshape_data(X)
        Y = self._reshape_data(Y)
        Z = self._reshape_data(Z)

        # Step 1: Linear regression of Y on Z
        Z_ones = np.column_stack((np.ones(Z.shape[0]), Z))
        self.beta_Z = np.linalg.pinv(Z_ones.T @ Z_ones) @ Z_ones.T @ Y
        Y_hat = Z_ones @ self.beta_Z
        e_Y = Y - Y_hat  # Residual of Y

        # Step 2: Regress the residual on X using the included regression model
        self.regression_model.fit(X, e_Y)

        # Obtain coefficients from the regression model
        if hasattr(self.regression_model, 'coef_'):
            self.beta_X = self.regression_model.coef_
        elif hasattr(self.regression_model, 'getBeta'):
            beta_output = self.regression_model.getBeta()
            self.beta_X = self._reshape_output(beta_output)

        # Obtain negative logarithm of p-values, if available
        if hasattr(self.regression_model, 'getNegLogP'):
            neg_log_p_output = self.regression_model.getNegLogP()
            if neg_log_p_output is not None:
                self.neg_log_p_values = self._reshape_output(neg_log_p_output)
                self.p_values = np.exp(-self.neg_log_p_values)
                # Concatenate the p-values of Z and X. The p-values of Z were not computed, mark with NaN.
                p_values_Z = np.full(Z.shape[1], np.nan)
                self.p_values = np.concatenate((p_values_Z, self.p_values))
    def predict(self, X, Z):
        X = self._reshape_data(X)
        Z = self._reshape_data(Z)

        Z_ones = np.column_stack((np.ones(Z.shape[0]), Z))
        ZX = np.column_stack((Z, X))
        combined_beta = np.concatenate((self.beta_Z[1:].ravel(), self.beta_X.ravel()))
        return ZX @ combined_beta + self.beta_Z[0]

    def get_coefficients(self):
        return np.concatenate((self.beta_Z[1:].ravel(), self.beta_X.ravel()))

    def get_p_values(self):
        return self.p_values


In [ ]:
from sklearn.metrics import accuracy_score, mean_squared_error


In [ ]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

In [ ]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

### 4. Discussion and report

In [ ]:
def interpret_result(model: Any, feature_names: List[str], trait: str, condition: str,
                     threshold: float = 0.05, save_output: bool = True,
                     output_dir: str = './output', model_id: int = 1) -> None:
    """This function interprets and reports the result of a trained linear regression model, where the regressor
    consists of one variable about condition and multiple variables about genetic factors.
    The function extracts coefficients and p-values from the model, and identifies the significant genes based on
    p-values or non-zero coefficients, depending on the availability of p-values.

    Parameters:
    model (Any): The trained regression Model.
    feature_names (List[str]): A list of feature names corresponding to the model's coefficients.
    trait (str): The target trait of interest.
    condition (str): The specific condition to examine within the model.
    threshold (float): Significance level for p-value correction. Defaults to 0.05.
    save_output (bool): Flag to determine whether to save the output to a file. Defaults to True.
    output_dir (str): Directory path where output files are saved. Defaults to './output'.
    model_id (int): The index of the model, 1 or 2.

    Returns:
    None: This function does not return anything but prints and optionally saves the output.
    """
    coefficients = model.get_coefficients().reshape(-1).tolist()
    p_values = model.get_p_values()
    if p_values is None:
        regression_df = pd.DataFrame({
            'Variable': feature_names,
            'Coefficient': coefficients
        })
    else:
        regression_df = pd.DataFrame({
            'Variable': feature_names,
            'Coefficient': coefficients,
            'p_value': p_values.reshape(-1).tolist()
        })
    condition_effect = regression_df[regression_df['Variable'] == condition].iloc[0]

    print(f"Effect of the condition on the target variable:")
    print(f"Variable: {condition}")
    print(f"Coefficient: {condition_effect['Coefficient']:.4f}")
    gene_regression_df = regression_df[regression_df['Variable'] != condition]
    if p_values is None:
        gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()
        significant_genes = gene_regression_df[gene_regression_df['Coefficient'] != 0]
        significant_genes_sorted = significant_genes.sort_values(by='Absolute Coefficient', ascending=False)
        print(
            f"Found {len(significant_genes_sorted)} genes with non-zero coefficients associated with the trait '{trait}' "
            f"conditional on the factor '{condition}'. These genes are identified as significant based on the regression model.")
    else:
        # Apply the Benjamini-Hochberg correction, to get the corrected p-values
        corrected_p_values = multipletests(gene_regression_df['p_value'], alpha=threshold, method='fdr_bh')[1]
        gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
        significant_genes = gene_regression_df.loc[gene_regression_df['corrected_p_value'] < threshold]
        significant_genes_sorted = significant_genes.sort_values('corrected_p_value')
        print(
            f"Found {len(significant_genes_sorted)} significant genes associated with the trait '{trait}' conditional on "
            f"the factor '{condition}', with corrected p-value < {threshold}:")

    print(significant_genes_sorted.to_string(index=False))

    # Optionally, save this to a CSV file
    if save_output:
        significant_genes_sorted.to_csv(
            os.path.join(output_dir, f'significant_genes_condition_{condition}_{model_id}.csv'), index=False)

In [ ]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

In [ ]:
from statsmodels.stats.multitest import multipletests


In [ ]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)